# Asset Data

In [1]:
import ast
import numpy as np
import pandas as pd
from warnings import filterwarnings
from tqdm.notebook import tqdm
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
from plotly.offline import iplot, init_notebook_mode
from pprint import pprint
from datetime import date

filterwarnings('ignore')

from src.contract_export import contract_query
from config import CONTRACT_ADDRESSES, NODE_LCD_URLS

init_notebook_mode(connected=True)
tqdm.pandas()

## load data

In [2]:
assets_df = pd.read_csv('data_csv/all_assets.csv', index_col=0)
assets_df['channels'] = assets_df.channels.map(lambda x: ast.literal_eval(x) if type(x) == str else None)
assets_df['denom_units'] = assets_df.denom_units.map(
    lambda x: ast.literal_eval(x) if type(x) == str else None)
assets_df

,chain_name,chain_id,denom,type_asset,supply,description,denom_units,display,name,symbol,...,denom_base,type_asset_base,path,channels,chain_id_counterparty,channel_id_counterparty,supply_base,chain_id_base,one_channel,admin
0,cronos,cronosmainnet_25-1,ibc/0FD310E1160487D42B93949F1D8955C7EF6D88B1BF...,ics20,500000000000001,NaN,None,NaN,NaN,NaN,...,acanto,sdk.coin,transfer/channel-9,[channel-9],canto_7700-1,channel-22,1055479087258475803746390781,canto_7700-1,True,NaN
1,cronos,cronosmainnet_25-1,ibc/11DFDFADE34DCE439BA732EBA5CD8AA804A544BA1E...,ics20,972901,Irishub staking token,"[{'denom': 'uiris', 'exponent': 0, 'aliases': ...",iris,NaN,NaN,...,uiris,sdk.coin,transfer/channel-8,[channel-8],irishub-1,channel-66,2159950917933764,irishub-1,True,NaN
2,cronos,cronosmainnet_25-1,ibc/2114969D9A69DD3F8529975297CA38AF474E34305D...,ics20,2,NaN,None,NaN,NaN,NaN,...,ucre,sdk.coin,transfer/channel-6,[channel-6],crescent-1,channel-39,393146039940308,crescent-1,True,NaN
3,cronos,cronosmainnet_25-1,ibc/4294C3DB67564CF4A0B2BFACC8415A59B38243F6FF...,ics20,1000000,The native stable token of the Terra Columbus.,"[{'denom': 'uusd', 'exponent': 0, 'aliases': [...",usd,USD TERRA,UST,...,uusd,sdk.coin,transfer/channel-2,[channel-2],columbus-5,channel-22,9794669019190595,columbus-5,True,NaN
4,cronos,cronosmainnet_25-1,ibc/4DAE26570FD24ABA40E2BE4137E39D946C78B00B24...,ics20,656142346920,NaN,None,NaN,NaN,NaN,...,uakt,sdk.coin,transfer/channel-4,[channel-4],akashnet-2,channel-40,217811008186244,akashnet-2,True,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12925,terra2,phoenix-1,cw20:terra17gck626vgax9jpe6utm7dhx4vdzawfkt0jh...,cw20,93000000000000,NaN,"[{'denom': 'dust', 'exponent': 6, 'aliases': [...",NaN,Depegged UST,dUST,...,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN
12926,terra2,phoenix-1,cw20:terra17aj4ty4sz4yhgm08na8drc0v03v2jwr3wax...,cw20,1434986596641,NaN,"[{'denom': 'bluna', 'exponent': 6, 'aliases': ...",NaN,boneLuna,bLUNA,...,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN
12927,terra2,phoenix-1,cw20:terra1xzkel96e5e8vfmqw7valzdzzv9hqasfyslc...,cw20,6572018665376,NaN,"[{'denom': 'local', 'exponent': 6, 'aliases': ...",NaN,Local Terra Token (Wormhole),LOCAL,...,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN
12928,terra2,phoenix-1,cw20:terra14xsm2wzvu7xaf567r693vgfkhmvfs08l68h...,cw20,347799217967,NaN,"[{'denom': 'lunax', 'exponent': 6, 'aliases': ...",NaN,Stader LunaX Token,LunaX,...,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN


## charts

In [3]:
def aggregate_pie_plot(assets_df: pd.DataFrame, agg_column: str, n: int = 8, object_name: str = 'items') -> pd.Series:
    assets_by_item_row = assets_df.groupby(agg_column)['denom'].agg(np.count_nonzero).sort_values(ascending=False)
    assets_by_item = assets_by_item_row.head(n)
    if len(assets_by_item_row) > n:
        assets_by_item[f'remaining {len(assets_by_item_row) - n:>,} {object_name}<br>'] = sum(assets_by_item_row[n:])
    return assets_by_item

In [4]:
ASSET_TYPE_DICT = \
    {'ics20': 'IBC (ICS-20)',
     'sdk.coin': 'Native',
     'pool': 'LP Tokens',
     'factory': 'Token Factory',
     'cw20': 'CW-20',
     'erc20': 'ERC-20'}

assets_by_chain = aggregate_pie_plot(assets_df=assets_df, agg_column='chain_name', object_name='chains\n', n=17)

assets_by_type_asset = aggregate_pie_plot(assets_df=assets_df, agg_column='type_asset')
assets_by_type_asset.index = assets_by_type_asset.index.map(lambda x: ASSET_TYPE_DICT[x])

assets_by_denom_base = aggregate_pie_plot(assets_df=assets_df, agg_column='denom_base', object_name='base denoms\n',
                                          n=6)

total_assets = len(assets_df)

In [5]:
fig = make_subplots(
    rows=2,
    cols=2,
    specs=[[{'type': 'pie', 'colspan': 2}, None],
           [{'type': 'pie'}, {'type': 'pie'}]],
    subplot_titles=('Cosmos-SDK Assets by Chains', 'by Type', 'by Base Denoms'),
    horizontal_spacing=0.4,
    vertical_spacing=0.14
)

fig.add_trace(
    go.Pie(
        labels=list(assets_by_chain.to_dict().keys()),
        values=list(assets_by_chain.to_dict().values()),
        text=list(assets_by_chain.map('{:>,}'.format).to_dict().values()),
        sort=False,
        name='assets by chains',
        rotation=-38,
        marker_colors=px.colors.qualitative.Plotly
    ),
    row=1,
    col=1)

fig.add_trace(
    go.Pie(
        labels=list(assets_by_type_asset.to_dict().keys()),
        values=list(assets_by_type_asset.to_dict().values()),
        text=list(assets_by_type_asset.map('{:>,}'.format).to_dict().values()),
        sort=False,
        name='assets by type',
        rotation=90,
        marker_colors=px.colors.qualitative.G10
    ),
    row=2,
    col=1)
fig.add_trace(
    go.Pie(
        labels=list(assets_by_denom_base.to_dict().keys()),
        values=list(assets_by_denom_base.to_dict().values()),
        text=list(assets_by_denom_base.map('{:>,}'.format).to_dict().values()),
        sort=False,
        name='assets by base denoms',
        rotation=75,
        marker_colors=px.colors.qualitative.Set1_r
    ),
    row=2,
    col=2)
fig.update_traces(
    textposition='outside',
    hoverinfo='label+value+percent',
    textinfo='label+value+percent',
    texttemplate=' %{label}  %{value:>,} (%{percent:.1%}) ',
    hovertemplate='%{label}<br>%{value:>,} (%{percent:.1%})'
)
fig.update_layout(
    height=850,
    width=1000,
    showlegend=False,
    font_family="Play",
    title_text='',
    title_x=0.5,
    template='plotly_white',
    autosize=False,
    margin={'l': 0, 'r': 0, 't': 80, 'b': 0}
)

fig['layout']['annotations'][0]['y'] = 1.04
fig['layout']['annotations'][1]['y'] = 0.37
fig['layout']['annotations'][2]['y'] = 0.37

fig.add_annotation(
    x=-0.3,
    y=-0.05,
    text=f'github.com/Snedashkovsky/on-chain-registry',
    showarrow=False)
fig.add_annotation(
    x=1.25,
    y=-0.05,
    text=f'updated {date.today()}',
    showarrow=False)

fig.write_image('charts/charts.png', scale=1.5)
fig.write_html('charts/index.html', include_plotlyjs='cdn')
iplot(fig)

## contract query

In [6]:
CONTRACT_CHAIN = 'osmosis-testnet'

pprint(
    contract_query(
        query={'get_assets_by_chain':
                   {'chain_name': 'osmosis',
                    'limit': 4,
                    'start_after_base': 'ibc/FF6F4774ABC2478832A6D6681DB9A7A8DDC4485212A8AED84642EFCD63C748A9'}
               },
        contract_address=CONTRACT_ADDRESSES[CONTRACT_CHAIN],
        node_lcd_url=NODE_LCD_URLS[CONTRACT_CHAIN]
    )
)

{'data': {'assets': [{'address': None,
                      'admin': None,
                      'base': 'ibc/FFA652599C77E853F017193E36B5AB2D4D9AFC4B54721A74904F80C9236BF3B7',
                      'chain_id': 'osmosis-1',
                      'chain_name': 'osmosis',
                      'coingecko_id': None,
                      'denom_units': [{'aliases': ['microlumen'],
                                       'denom': 'ulumen',
                                       'exponent': 0},
                                      {'aliases': ['millilumen'],
                                       'denom': 'mlumen',
                                       'exponent': 3},
                                      {'aliases': ['lumen'],
                                       'denom': 'lumen',
                                       'exponent': 6}],
                      'description': 'Demon units of the LumenX',
                      'display': 'lumen',
                      'ibc': {'base_supply':

In [7]:
pprint(
    contract_query(
        query={'get_all_assets':
                   {'limit': 5}
               },
        contract_address=CONTRACT_ADDRESSES[CONTRACT_CHAIN],
        node_lcd_url=NODE_LCD_URLS[CONTRACT_CHAIN]
    )
)

{'data': {'entries': [{'address': None,
                       'admin': None,
                       'base': 'ibc/4A3AAD07BC4EBEBC10FC2560EAA3B7A1D3B541B5264ED8E5E13E6B74AC76127B',
                       'chain_id': 'eightball-1',
                       'chain_name': '8ball',
                       'coingecko_id': None,
                       'denom_units': [{'aliases': [],
                                        'denom': 'uosmo',
                                        'exponent': 0},
                                       {'aliases': [],
                                        'denom': 'osmo',
                                        'exponent': 6}],
                       'description': 'The native token of Osmosis',
                       'display': 'osmo',
                       'ibc': {'base_supply': None,
                               'dst_channel': 'channel-641',
                               'source_channel': 'channel-16',
                               'source_denom': 'uosmo

In [8]:
pprint(
    contract_query(
        query={'get_asset':
                   {'chain_name': 'osmosis',
                    'base': 'uosmo'}
               },
        contract_address=CONTRACT_ADDRESSES[CONTRACT_CHAIN],
        node_lcd_url=NODE_LCD_URLS[CONTRACT_CHAIN]
    )
)

{'data': {'asset': {'address': None,
                    'admin': None,
                    'base': 'uosmo',
                    'chain_id': 'osmosis-1',
                    'chain_name': 'osmosis',
                    'coingecko_id': None,
                    'denom_units': [{'aliases': [],
                                     'denom': 'uosmo',
                                     'exponent': 0},
                                    {'aliases': [],
                                     'denom': 'osmo',
                                     'exponent': 6}],
                    'description': 'The native token of Osmosis',
                    'display': 'osmo',
                    'ibc': None,
                    'images': None,
                    'keywords': None,
                    'logo_uris': None,
                    'name': None,
                    'supply': '613859539497497',
                    'symbol': None,
                    'traces': None,
                    'type_asset':